In [83]:
import pandas as pd
import torch as T
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

print ('Available devices', T.cuda.device_count())
device = T.device('cuda' if T.cuda.is_available() else 'cpu')
print("Device:", device)
T.cuda.get_device_name(0)

Available devices 1
Device: cuda


'NVIDIA GeForce GTX 1080 Ti'

In [84]:
df = pd.read_csv('match_data_v5.csv').drop('last', axis=1)

In [88]:
X = df.drop(['blueWin', 'matchID'], axis=1)
y = df['blueWin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = T.tensor(X_train.values, dtype=T.float32, device=device)
X_test = T.tensor(X_test.values, dtype=T.float32, device=device)
y_train = T.tensor(y_train.values, dtype=T.float32, device=device).view(-1, 1)
y_test = T.tensor(y_test.values, dtype=T.float32, device=device).view(-1, 1)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 128

# Create DataLoader instances for training and test sets
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  # No need to shuffle test data

In [93]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1     = nn.Linear(in_features=27, out_features=512)
        self.norm1   = nn.BatchNorm1d(512, momentum=0.9)

        self.fc2     = nn.Linear(in_features=512, out_features=256)
        self.norm2   = nn.BatchNorm1d(256, momentum=0.8)

        self.fc3     = nn.Linear(in_features=256, out_features=128)
        self.norm3   = nn.BatchNorm1d(128, momentum=0.7)

        self.fc4     = nn.Linear(in_features=128, out_features=32)
        self.norm4   = nn.BatchNorm1d(32, momentum=0.7)

        self.act     = nn.ReLU()
        self.output  = nn.Linear(in_features=32, out_features=1)

        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.act(self.norm1(self.fc1(x)))
        x = self.act(self.norm2(self.fc2(x)))
        x = self.act(self.norm3(self.fc3(x)))
        x = self.act(self.norm4(self.fc4(x)))
        x = self.sigmoid(self.output(x))
        return x

In [94]:
criterion = nn.BCELoss()
model = Model().to(device=device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    total_loss = 0
    for x_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        logits = model( x_batch )
        loss = criterion( logits, y_batch )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"epoch={epoch} Training Loss: {round(total_loss, 5)}")

epoch=0 Training Loss: 76.51149
epoch=1 Training Loss: 75.75999
epoch=2 Training Loss: 75.61533
epoch=3 Training Loss: 75.33131
epoch=4 Training Loss: 75.26352
epoch=5 Training Loss: 75.07919
epoch=6 Training Loss: 75.20025
epoch=7 Training Loss: 75.07555
epoch=8 Training Loss: 75.22798
epoch=9 Training Loss: 74.97421
epoch=10 Training Loss: 75.1411
epoch=11 Training Loss: 74.78207
epoch=12 Training Loss: 74.73398
epoch=13 Training Loss: 74.87786
epoch=14 Training Loss: 74.88714
epoch=15 Training Loss: 74.73016
epoch=16 Training Loss: 74.57953
epoch=17 Training Loss: 74.76304
epoch=18 Training Loss: 74.8326
epoch=19 Training Loss: 74.70199
epoch=20 Training Loss: 74.66279
epoch=21 Training Loss: 74.55357
epoch=22 Training Loss: 74.51359
epoch=23 Training Loss: 74.43455
epoch=24 Training Loss: 74.49512
epoch=25 Training Loss: 74.27851
epoch=26 Training Loss: 74.37236
epoch=27 Training Loss: 74.21251
epoch=28 Training Loss: 74.25828
epoch=29 Training Loss: 74.08119
epoch=30 Training Loss

In [95]:
from sklearn.metrics import r2_score

predictions_np = model( X_test ).detach().round().cpu().numpy()
r2_score(predictions_np, y_test.cpu().numpy())

-0.09416960961045628